In [33]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [34]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df=application_df.drop(["EIN", "NAME"], axis='columns')

In [35]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
n = application_df.nunique(axis=0)
  
print("Number of unique values in each column :\n",
      n)

Number of unique values in each column :
 APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [37]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
application_types_counts=application_df['APPLICATION_TYPE'].value_counts()
application_types_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [38]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

application_types_to_replace=list(application_types_counts.index[8:17])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [39]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
classification_counts=application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1728        1
C1570        1
C2380        1
C2600        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [40]:
# You may find it helpful to look at CLASSIFICATION value counts >20
#  YOUR CODE GOES HERE
classification_counts[classification_counts>100]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [41]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace=list(classification_counts[classification_counts<104].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [43]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
categorical_columns=['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION','USE_CASE','ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']
application_df_categorical=pd.get_dummies(application_df, columns=categorical_columns)

application_df_categorical.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [44]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df_categorical.drop('IS_SUCCESSFUL', axis=1).values
y = application_df_categorical['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [45]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [68]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  300
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))
# Second hidden layer
#  YOUR CODE GOES HERE
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 300)               15300     
                                                                 
 dense_5 (Dense)             (None, 100)               30100     
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                                                                 
Total params: 45,501
Trainable params: 45,501
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [70]:
#os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weight.{epoch:02d}h5"

In [71]:
# Create a callback to save weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=0,
    save_weights_only=True,
    save_freq=5)


Trial #1 try epochs, layer#1 - 300 neurons, layer#2 100 neurons 

In [72]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled,y_train,epochs=20, callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5738 - accuracy: 0.7187
Epoch 2/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5566 - accuracy: 0.7255 1s - loss: 0.5556 - accuracy:  -
Epoch 3/20
804/804 [==============================] - 9s 11ms/step - loss: 0.5524 - accuracy: 0.7283
Epoch 4/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5493 - accuracy: 0.7289 3s - loss: 0.5
Epoch 5/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5486 - accuracy: 0.7315
Epoch 6/20
804/804 [==============================] - 9s 11ms/step - loss: 0.5469 - accuracy: 0.7308 1s - loss: 0.5469  - ETA: 0s - loss: 0
Epoch 7/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5456 - accuracy: 0.7319
Epoch 8/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5451 - accuracy: 0.7340
Epoch 9/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5444 - accuracy: 0.7328 1s
E

In [73]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5526 - accuracy: 0.7291 - 322ms/epoch - 1ms/step
Loss: 0.5526420474052429, Accuracy: 0.7290962338447571


Trial #2 - create optimised method

In [63]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [64]:
# Import the kerastuner library
import keras_tuner as kt
import os

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    directory=os.path.normpath('C:/untitled_project/07'),
    hyperband_iterations=2)

In [65]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 29s]
val_accuracy: 0.7292128205299377

Best val_accuracy So Far: 0.7341107726097107
Total elapsed time: 00h 11m 38s
INFO:tensorflow:Oracle triggered exit


In [66]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 99,
 'num_layers': 2,
 'units_0': 15,
 'units_1': 9,
 'units_2': 5,
 'units_3': 5,
 'units_4': 25,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': 'f7fcf711c42e988ae937bfb0557ff31f'}

In [67]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
best_model.save('Sequential007.h5')
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5505 - accuracy: 0.7341 - 265ms/epoch - 988us/step
Loss: 0.5504863262176514, Accuracy: 0.7341107726097107


In [74]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
best_model.save("AlphabetSoupCharity_Optimization.h5")

Trial #3 - reduce number of input parameters to a minimum

In [75]:
application_df = pd.read_csv("Resources/charity_data.csv")

In [76]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

application_types_to_replace=list(application_types_counts.index[6:17])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
Other     1529
T6        1216
T5        1173
T19       1065
T8         737
Name: APPLICATION_TYPE, dtype: int64

In [77]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df=application_df.drop(["EIN", "NAME"], axis='columns')

In [78]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace=list(classification_counts[classification_counts<2000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [79]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
categorical_columns=['APPLICATION_TYPE','AFFILIATION', 'CLASSIFICATION','USE_CASE','ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']
application_df_categorical=pd.get_dummies(application_df, columns=categorical_columns)

application_df_categorical.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [80]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = application_df_categorical.drop('IS_SUCCESSFUL', axis=1).values
y = application_df_categorical['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [81]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [82]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  300
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 10
nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))
# Second hidden layer
#  YOUR CODE GOES HERE
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))
# Output layer
#  YOUR CODE GOES HERE
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 300)               12300     
                                                                 
 dense_8 (Dense)             (None, 100)               30100     
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 42,501
Trainable params: 42,501
Non-trainable params: 0
_________________________________________________________________


In [83]:
# Compile the model
#  YOUR CODE GOES HERE
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [84]:
# Train the model
#  YOUR CODE GOES HERE
fit_model = nn.fit(X_train_scaled,y_train,epochs=20, callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 8s 9ms/step - loss: 0.5723 - accuracy: 0.7204
Epoch 2/20
804/804 [==============================] - 7s 9ms/step - loss: 0.5603 - accuracy: 0.7274
Epoch 3/20
804/804 [==============================] - 8s 9ms/step - loss: 0.5572 - accuracy: 0.7290: 1s - loss: 0.5592 - accuracy - ETA: 1s - loss: 0.5593 - accuracy: 0. - ETA: 0s - loss: 0
Epoch 4/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5551 - accuracy: 0.7280
Epoch 5/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5550 - accuracy: 0.7294
Epoch 6/20
804/804 [==============================] - 7s 9ms/step - loss: 0.5535 - accuracy: 0.7301
Epoch 7/20
804/804 [==============================] - 7s 9ms/step - loss: 0.5519 - accuracy: 0.7297
Epoch 8/20
804/804 [==============================] - 8s 10ms/step - loss: 0.5520 - accuracy: 0.7313 1s
Epoch 9/20
804/804 [==============================] - 7s 9ms/step - loss: 0.5511 - accuracy: 0.7318
Epoc

In [85]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5580 - accuracy: 0.7263 - 311ms/epoch - 1ms/step
Loss: 0.5579825639724731, Accuracy: 0.7262973785400391


Report:




The purpose of the analysis is to fit a predictive model to evaluate success rate of funded entities based on a set of input parameters. 


Analysis yielded the following results:



*Data Preprocessing

The targe of the model is IS_SUCCESSFUL

The features of the model are the rest of the variables except EIN and entity name. 

EIN and entity name were remuved from consideration. 

* Compiling, Training, and Evaluating the Model

60-300 neurons and up to three activation layers were tried. In one trial optimization scheve was used.
Target performance of 0.75 accurace was not achieved
Reducing number of epocs from 100 to 20 improved model computation speed without reducing the accuracy. 





Summary: Three approaches yoielded similar prediction accuracy of 0.72-0.73

